In [1]:
import pandas as pd
import numpy as np

In [2]:
-0.6548
-0.6317
-0.3686
-0.1090
5
0.0539
6
293.6831
62
0.2412

0.2412

In [3]:
water = pd.read_csv('water.txt', sep = '\t')
water.head()

,location,town,mortality,hardness
0,South,Bath,1247,105
1,North,Birkenhead,1668,17
2,South,Birmingham,1466,5
3,North,Blackburn,1800,14
4,North,Blackpool,1609,18


In [4]:
mortality = water.mortality.values
hardness = water.hardness.values

In [5]:
sum_1 = 0
sum_2 = 0
sum_3 = 0

for mort, hard in zip(mortality, hardness):
    sum_1 += (mort - np.mean(mortality)) * (hard - np.mean(hardness))
    sum_2 += (mort - np.mean(mortality)) ** 2
    sum_3 += (hard - np.mean(hardness)) ** 2
    
print sum_1 / (sum_2 * sum_3) ** 0.5

-0.6548486232042464


In [6]:
from scipy.stats import pearsonr
pearsonr(mortality, hardness)

(-0.6548486232042465, 1.0331339260349571e-08)

In [7]:
from scipy.stats import spearmanr
spearmanr(mortality, hardness)

SpearmanrResult(correlation=-0.6316646189166502, pvalue=4.79546153722838e-08)

In [8]:
south = water[water.location == 'South']
south_pear = pearsonr(south.mortality.values, south.hardness.values)

north = water[water.location == 'North']
north_pear = pearsonr(north.mortality.values, north.hardness.values)

print south_pear
print north_pear

(-0.6021532715484158, 0.0011346743048161927)
(-0.3685978383288718, 0.029344659047110606)


In [9]:
a = 203
b = 718
c = 239
d = 515
print (a * d - b * c) / ((a + b) * (a + c) * (b + d) * (d + c)) ** 0.5
from sklearn.metrics import matthews_corrcoef 
print matthews_corrcoef([a, c], [b, d])

-0.109002374587
0.0


In [10]:
from scipy.stats import chi2_contingency
chi2_contingency([[a, b], [c, d]])

(19.40753078854304,
 1.0558987006638725e-05,
 1,
 array([[243.03402985, 677.96597015],
        [198.96597015, 555.03402985]]))

In [11]:
chi2_contingency([[48, 30], [92, 36]])

(1.9271792226870357, 0.16506738276246485, 1, array([[53.00970874, 24.99029126],
        [86.99029126, 41.00970874]]))

In [12]:
chi2_contingency([[197, 111, 33], [382, 685, 331], [110, 342, 333]])

(293.68311039689746,
 2.4964299580093467e-62,
 4,
 array([[ 93.08597464, 153.74722662,  94.16679873],
        [381.6251981 , 630.318542  , 386.0562599 ],
        [214.28882726, 353.93423138, 216.77694136]]))

In [13]:
import scipy.stats as ss
def cramers_corrected_stat(confusion_matrix):
    """ calculate Cramers V statistic for categorial-categorial association.
        uses correction from Bergsma and Wicher, 
        Journal of the Korean Statistical Society 42 (2013): 323-328
    """
    chi2 = ss.chi2_contingency(confusion_matrix)[0]
    n = confusion_matrix.sum()
    phi2 = chi2/n
    r,k = confusion_matrix.shape
    phi2corr = max(0, phi2 - ((k-1)*(r-1))/(n-1))    
    rcorr = r - ((r-1)**2)/(n-1)
    kcorr = k - ((k-1)**2)/(n-1)
    return np.sqrt(phi2corr / min( (kcorr-1), (rcorr-1)))

In [14]:
cramers_corrected_stat(np.array([[197, 111, 33], [382, 685, 331], [110, 342, 333]]))

0.2412013934500338

In [15]:
import scipy

fem = np.append(np.ones(203), np.zeros(718))
mal = np.append(np.ones(239), np.zeros(515))

In [16]:
def proportions_diff_confint_ind(sample1, sample2, alpha = 0.05):    
    z = scipy.stats.norm.ppf(1 - alpha / 2.)
    
    p1 = float(sum(sample1)) / len(sample1)
    p2 = float(sum(sample2)) / len(sample2)
    
    left_boundary = (p1 - p2) - z * np.sqrt(p1 * (1 - p1)/ len(sample1) + p2 * (1 - p2)/ len(sample2))
    right_boundary = (p1 - p2) + z * np.sqrt(p1 * (1 - p1)/ len(sample1) + p2 * (1 - p2)/ len(sample2))
    
    return (left_boundary, right_boundary)

def proportions_diff_z_stat_ind(sample1, sample2):
    n1 = len(sample1)
    n2 = len(sample2)
    
    p1 = float(sum(sample1)) / n1
    p2 = float(sum(sample2)) / n2 
    P = float(p1*n1 + p2*n2) / (n1 + n2)
    
    return (p1 - p2) / np.sqrt(P * (1 - P) * (1. / n1 + 1. / n2))

def proportions_diff_z_test(z_stat, alternative = 'two-sided'):
    if alternative not in ('two-sided', 'less', 'greater'):
        raise ValueError("alternative not recognized\n"
                         "should be 'two-sided', 'less' or 'greater'")
    
    if alternative == 'two-sided':
        return 2 * (1 - scipy.stats.norm.cdf(np.abs(z_stat)))
    
    if alternative == 'less':
        return scipy.stats.norm.cdf(z_stat)

    if alternative == 'greater':
        return 1 - scipy.stats.norm.cdf(z_stat)

In [17]:
print proportions_diff_confint_ind(mal, fem)

(0.053905233215813156, 0.13922183141523897)


In [18]:
proportions_diff_z_test(proportions_diff_z_stat_ind(mal, fem))

8.153453089576601e-06

# Test 2

Выберите задачи, в которых обязательно нужно применять поправку на множественную проверку гипотез

* Сравнение эффективности лечения пациентов в подгруппах по большому количеству признаков

* Выбор инвестиционных фондов с помощью сравнения доходности каждого из них с доходностью базового актива по нескольким историческим периодам

* Попарное сравнение средних большого количества выборок

* Локализация различий в активности мозга испытуемых в разных экспериментальных условиях

Классификатор C4.5 и три его модификации: с оптимизацией гиперпараметра m, гиперпараметра cf и с одновременной оптимизацией обоих гиперпараметров. Эти четыре классификатора сравнивались на 14 наборах данных. На каждом датасете был посчитан AUC каждого классификатора. Данные записаны в файле:
        Используя критерий знаковых рангов, проведите попарное сравнение каждого классификатора с каждым. Выберите два классификатора, различие между которыми наиболее статистически значимо.

In [22]:
aucs = pd.read_csv('AUCs.txt', sep = '\t')
aucs.head()

,Unnamed: 0,C4.5,C4.5+m,C4.5+cf,C4.5+m+cf
0,adult (sample),0.763,0.768,0.771,0.798
1,breast cancer,0.599,0.591,0.590,0.569
2,breast cancer wisconsin,0.954,0.971,0.968,0.967
3,cmc,0.628,0.661,0.654,0.657
4,ionosphere,0.882,0.888,0.886,0.898


In [59]:
from scipy.stats import wilcoxon
from itertools import combinations
result = 10000000
answer = ''

counter = 0
p = []
for comb in combinations(['C4.5', 'C4.5+m', 'C4.5+cf', 'C4.5+m+cf'], 2):
    cur_result = wilcoxon(- aucs[comb[0]] + aucs[comb[1]])
    print cur_result, comb
    if cur_result.statistic < result:
        result = cur_result.statistic
        answer = comb
    if cur_result.pvalue < 0.05:
        counter += 1
    p.append(cur_result.pvalue)
print answer

WilcoxonResult(statistic=6.5, pvalue=0.01075713311978963) ('C4.5', 'C4.5+m')
WilcoxonResult(statistic=43.0, pvalue=0.861262330095348) ('C4.5', 'C4.5+cf')
WilcoxonResult(statistic=11.0, pvalue=0.015906444101703374) ('C4.5', 'C4.5+m+cf')
WilcoxonResult(statistic=17.0, pvalue=0.046332729793395394) ('C4.5+m', 'C4.5+cf')
WilcoxonResult(statistic=22.0, pvalue=0.3278256758446406) ('C4.5+m', 'C4.5+m+cf')
WilcoxonResult(statistic=10.0, pvalue=0.022909099354356588) ('C4.5+cf', 'C4.5+m+cf')
('C4.5', 'C4.5+m')


Сколько статистически значимых на уровне 0.05 различий мы обнаружили?

In [60]:
counter

4

Судя по данным из предыдущего опроса, настройка какого из параметров классификатора даёт более значимое увеличение качества?

Сравнивая 4 классификатора между собой, мы проверили 6 гипотез. Давайте сделаем поправку на множественную проверку. Начнём с метода Холма. Сколько гипотез можно отвергнуть на уровне значимости 0.05 после поправки этим методом?

In [57]:
from statsmodels.sandbox.stats.multicomp import multipletests 
reject, p_corrected, a1, a2 = multipletests(p, 
                                            alpha = 0.05, 
                                            method = 'holm') 
print reject

[False False False False False False]


Сколько гипотез можно отвергнуть на уровне значимости 0.05 после поправки методом Бенджамини-Хохберга?

In [58]:
from statsmodels.sandbox.stats.multicomp import multipletests 
reject, p_corrected, a1, a2 = multipletests(p, 
                                            alpha = 0.05, 
                                            method = 'fdr_bh') 
print reject

[ True False  True False False  True]


Насколько корректно, на ваш взгляд, применение метода Бенджамини-Хохберга в этой задаче?

Всегда корректно, это же метод Бенджамини-Хохберга

* Некорректно: статистики, соответствующие разным гипотезам, зависимы, так что предположения метода Бенджамини-Хохберга не выполняются

Корректно: статистики, соответствующие разным гипотезам, независимы, так что предположения метода Бенджамини-Хохберга выполняются

Мы подозреваем, что в проведённом с C4.5 эксперименте на самом деле классификаторы сильнее отличаются друг от друга, просто нам не удалось это заметить. Что можно сделать, чтобы увеличить вероятность обнаружения различий, если они действительно существуют?

* Взять больше датасетов

Закрыть глаза на эффект множественной проверки гипотез и не делать никакой поправки

Попробовать настроить больше гиперпараметро